In [ ]:
# default_exp parser

In [ ]:
#export
import re

# Parser
class to parse out the [tegrastats](https://docs.nvidia.com/jetson/archives/l4t-archived/l4t-3231/index.html#page/Tegra%20Linux%20Driver%20Package%20Development%20Guide/AppendixTegraStats.html) information into a dict


In [ ]:
#export
class ParseTegrastats:
    """got most of this code from: https://github.com/ssaraff98/tegrastats_parser/blob/main/parse.py"""
    @staticmethod
    def parse_ram(lookup_table, ram):
        lookup_table['Used RAM (MB)'] = float(ram[0])
        lookup_table['Total RAM (MB)'] = float(ram[1])
        lookup_table['Number of Free RAM Blocks'] = float(ram[2])
        lookup_table['Size of Free RAM Blocks (MB)'] = float(ram[3])
        return lookup_table
    
    @staticmethod
    def parse_swap(lookup_table, swap):
        lookup_table['Used SWAP (MB)'] = float(swap[0])
        lookup_table['Total SWAP (MB)'] = float(swap[1])
        lookup_table['Cached SWAP (MB)'] = float(swap[2])
        return lookup_table

    @staticmethod
    def parse_iram(lookup_table, iram):
        lookup_table['Used IRAM (kB)'] = float(iram[0])
        lookup_table['Total IRAM (kB)'] = float(iram[1])
        lookup_table['Size of IRAM Blocks (kB)'] = float(iram[2])
        return lookup_table

    @staticmethod
    def parse_cpus(lookup_table, cpus):
        frequency = re.findall(r'@([0-9]*)', cpus)
        lookup_table['CPU Frequency (MHz)'] = float(frequency[0]) if frequency else ''
        for i, cpu in enumerate(cpus.split(',')):
            lookup_table[f'CPU {i} Load (%)'] = cpu.split('%')[0]
        return lookup_table

    @staticmethod
    def parse_gr3d(lookup_table, gr3d):
        lookup_table['Used GR3D (%)'] = float(gr3d[0])
        lookup_table['GR3D Frequency (MHz)'] = float(gr3d[1]) if gr3d[1] else ''
        return lookup_table

    @staticmethod
    def parse_emc(lookup_table, emc):
        lookup_table['Used EMC (%)'] = float(emc[0])
        lookup_table['GR3D Frequency (MHz)'] = float(emc[1])  if emc[1] else ''
        return lookup_table

    @staticmethod
    def parse_temperatures(lookup_table, temperatures):
        for label, temperature in temperatures:
            lookup_table[f'{label} Temperature (C)'] = float(temperature)
        return lookup_table

    @staticmethod
    def parse_vdds(lookup_table, vdds):
        for label, curr_vdd, avg_vdd in vdds:
            lookup_table[f'Current {label} Power Consumption (mW)'] = float(curr_vdd)
            lookup_table[f'Average {label} Power Consumption (mW)'] = float(avg_vdd)
        return lookup_table

    @staticmethod
    def parse_data(line):
        lookup_table = {}

        ram = re.findall(r'RAM ([0-9]*)\/([0-9]*)MB \(lfb ([0-9]*)x([0-9]*)MB\)', line)
        ParseTegrastats.parse_ram(lookup_table, ram[0]) if ram else None

        swap = re.findall(r'SWAP ([0-9]*)\/([0-9]*)MB \(cached ([0-9]*)MB\)', line)
        ParseTegrastats.parse_swap(lookup_table, swap[0]) if swap else None

        iram = re.findall(r'IRAM ([0-9]*)\/([0-9]*)kB \(lfb ([0-9]*)kB\)', line)
        ParseTegrastats.parse_iram(lookup_table, iram[0]) if iram else None

        cpus = re.findall(r'CPU \[(.*)\]', line)
        ParseTegrastats.parse_cpus(lookup_table, cpus[0]) if cpus else None

        ape = re.findall(r'APE ([0-9]*)', line)
        if ape:
            lookup_table['APE frequency (MHz)'] = float(ape[0])

        gr3d = re.findall(r'GR3D_FREQ ([0-9]*)%@?([0-9]*)?', line)
        ParseTegrastats.parse_gr3d(lookup_table, gr3d[0]) if gr3d else None

        emc = re.findall(r'EMC_FREQ ([0-9]*)%@?([0-9]*)?', line)
        ParseTegrastats.parse_emc(lookup_table, emc[0]) if emc else None

        nvenc = re.findall(r'NVENC ([0-9]*)', line)
        if nvenc:
            lookup_table['NVENC frequency (MHz)'] = float(nvenc[0])

        mts = re.findall(r'MTS fg ([0-9]*)% bg ([0-9]*)%', line) # !!!!

        temperatures = re.findall(r'([A-Za-z]*)@([0-9.]*)C', line)
        vdds = None

        if temperatures:
            ParseTegrastats.parse_temperatures(lookup_table, temperatures)
            substring = line[(line.rindex(temperatures[-1][1] + "C") + len(temperatures[-1][1] + "C")):]

            vdds = re.findall(r'([A-Za-z0-9_]*) ([0-9]*)\/([0-9]*)', substring)

        else:
            vdds = re.findall(r'VDD_([A-Za-z0-9_]*) ([0-9]*)\/([0-9]*)', line)
        ParseTegrastats.parse_vdds(lookup_table, vdds) if vdds else None

        return lookup_table

    @staticmethod
    def parse_line(line):
        return ParseTegrastats.parse_data(line)

In [ ]:
# example line from tegrastats
line = "RAM 1903/15692MB (lfb 3251x4MB) CPU [1%@1190,1%@1190,2%@1190,0%@1190,0%@1190,0%@1190,0%@1190,0%@1190] EMC_FREQ 0% GR3D_FREQ 0% AO@32.5C GPU@32C Tboard@32C Tdiode@34.75C AUX@31.5C CPU@33.5C thermal@32.9C PMIC@100C GPU 0/0 CPU 216/216 SOC 864/864 CV 0/0 VDDRQ 144/144 SYS5V 1889/1889"

In [ ]:
d = ParseTegrastats.parse_line(line)
d

{'Used RAM (MB)': 1903.0,
 'Total RAM (MB)': 15692.0,
 'Number of Free RAM Blocks': 3251.0,
 'Size of Free RAM Blocks (MB)': 4.0,
 'CPU Frequency (MHz)': 1190.0,
 'CPU 0 Load (%)': '1',
 'CPU 1 Load (%)': '1',
 'CPU 2 Load (%)': '2',
 'CPU 3 Load (%)': '0',
 'CPU 4 Load (%)': '0',
 'CPU 5 Load (%)': '0',
 'CPU 6 Load (%)': '0',
 'CPU 7 Load (%)': '0',
 'Used GR3D (%)': 0.0,
 'GR3D Frequency (MHz)': '',
 'Used EMC (%)': 0.0,
 'AO Temperature (C)': 32.5,
 'GPU Temperature (C)': 32.0,
 'Tboard Temperature (C)': 32.0,
 'Tdiode Temperature (C)': 34.75,
 'AUX Temperature (C)': 31.5,
 'CPU Temperature (C)': 33.5,
 'thermal Temperature (C)': 32.9,
 'PMIC Temperature (C)': 100.0,
 'Current GPU Power Consumption (mW)': 0.0,
 'Average GPU Power Consumption (mW)': 0.0,
 'Current CPU Power Consumption (mW)': 216.0,
 'Average CPU Power Consumption (mW)': 216.0,
 'Current SOC Power Consumption (mW)': 864.0,
 'Average SOC Power Consumption (mW)': 864.0,
 'Current CV Power Consumption (mW)': 0.0,
 'Ave

In [ ]:
assert type(d) is dict
assert "Used RAM (MB)" in d.keys()
assert len(d.keys())==36

In [ ]:
%timeit ParseTegrastats.parse_line(line)

59 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
from nbdev.export import *
notebook2script()